In [8]:
with open('text.txt','r',encoding='utf-8') as file:
    full_text = file.read()

In [9]:
import yake
kw_extractor = yake.KeywordExtractor(top=10, stopwords=None)
keywords = kw_extractor.extract_keywords(full_text)
for kw, v in keywords:
  print("Keyphrase: ",kw, ": score", v)

Keyphrase:  problem : score 0.033776429698420395
Keyphrase:  solution : score 0.03400267747642423
Keyphrase:  constraint : score 0.035497958364960655
Keyphrase:  KKT : score 0.04732586034227692
Keyphrase:  inequality constraints : score 0.05978889522173199
Keyphrase:  solution ifthat constraint : score 0.07671259011733038
Keyphrase:  constraint excludesother solutions : score 0.07934413620739232
Keyphrase:  gradient : score 0.08665319518106154
Keyphrase:  generalized La-grange function : score 0.08694292445211135
Keyphrase:  CHAPTER : score 0.09706171580819022


In [10]:
from multi_rake import Rake
rake = Rake()
keywords = rake.apply(full_text)
print(keywords[:10])

[('deﬁne itwith addition', 9.0), ('a\ue03ea + 2λi', 9.0), ('develop machinelearning algorithms', 9.0), ('constrained opti-mization problems', 8.5), ('=12||ax − b||22', 8.5), ('localstationary points excluded', 8.333333333333334), ('becausethe true function', 8.333333333333334), ('taking small steps', 8.0), ('quadratic approximation employed', 8.0), ('linear equation forxwill', 8.0)]


In [11]:
from summa import keywords
TR_keywords = keywords.keywords(full_text, scores=True)
print(TR_keywords[0:10])

[('x', 0.40313281614177177), ('problem', 0.2261230157809111), ('problems', 0.2261230157809111), ('α', 0.19620608475562126), ('constraints', 0.1892540852100651), ('constraint', 0.1892540852100651), ('gradient', 0.17889355923585482), ('points', 0.15225349414764922), ('point', 0.15225349414764922), ('norm', 0.1506210371743589)]


In [6]:
from keybert import KeyBERT
kw_model = KeyBERT(model='all-mpnet-base-v2')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 1.18k/1.18k [00:00<00:00, 350kB/s]
Downloading: 100%|██████████| 190/190 [00:00<00:00, 65.5kB/s]
Downloading: 100%|██████████| 10.6k/10.6k [00:00<00:00, 2.65MB/s]
Downloading: 100%|██████████| 116/116 [00:00<00:00, 43.6kB/s]
Downloading: 100%|██████████| 39.3k/39.3k [00:00<00:00, 126kB/s] 
Downloading: 100%|██████████| 438M/438M [00:29<00:00, 14.9MB/s] 
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 13.0kB/s]
Downloading: 100%|██████████| 239/239 [00:00<00:00, 79.6kB/s]
Downloading: 100%|██████████| 466k/466k [00:01<00:00, 331kB/s]  
Downloading: 100%|██████████| 363/363 [00:00<00:00, 117kB/s]
Downloading: 100%|██████████| 13.1k/13.1k [00:00<00:00, 2.56MB/s]
Downloading

['singular value decomposition', 'eigendecomposition involves analyzing', 'eigendecomposition', 'square eigendecomposition deﬁned', 'eigendecomposition functions', 'eigendecomposition involves', 'eigendecomposition deﬁned', 'ofthe eigendecomposition', 'svd', 'square eigendecomposition']


In [12]:
keywords = kw_model.extract_keywords(full_text, 

                                     keyphrase_ngram_range=(1, 3), 

                                     stop_words='english', 

                                     highlight=False,

                                     top_n=10)

keywords_list= list(dict(keywords).keys())

print(keywords_list)